In [37]:
import os 
import getopt
import sys

import random
import re

import numpy as np
import h5py
import pickle
import random
import copy
import pandas as pd
import math 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Masking, Lambda, concatenate, Bidirectional, Dense, Dropout, Flatten, Conv1D,BatchNormalization,  MaxPooling1D, Bidirectional, GRU, TimeDistributed
import tensorflow as tf
from tensorflow import keras



np.random.seed(1337) # for reproducibility
vocab = ["a", "g", "c", "t"]
indices = tf.range(len(vocab), dtype = tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab,indices)
table = tf.lookup.StaticVocabularyTable(table_init, 1)
record_defaults = [
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
]


In [38]:

# We end up not using this, it confuses the model to see different sequences each time

def replace_zeros_with_random_one(input_tensor):
    zero_mask = tf.reduce_all(tf.equal(input_tensor, 0), axis=-1)
    random_values = tf.random.uniform(shape=(tf.reduce_sum(tf.cast(zero_mask, tf.int32)),), maxval=input_tensor.shape[-1], dtype=tf.int64)

    # Create indices for replacement
    indices = tf.where(zero_mask)

    # Replace zeros with ones at random indices
    input_tensor = tf.tensor_scatter_nd_update(input_tensor, indices, tf.one_hot(random_values, depth=input_tensor.shape[-1]))

    return input_tensor

input_tensor = tf.constant([
    [[0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 1, 0]],
    [[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]]
], dtype=tf.float32)

output_tensor = replace_zeros_with_random_one(input_tensor)
print(input_tensor)
print(output_tensor)


tf.Tensor(
[[[0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]], shape=(2, 3, 5), dtype=float32)
tf.Tensor(
[[[0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 1. 0.]]], shape=(2, 3, 5), dtype=float32)


In [39]:

# Nadav dataset

def data_reader(file, batch_size=100, n_parse_threads=4):
    """Method for reading the data in an optimized way, can be used inside model.fit()
    
    Args:
        file (_type_): path to csv file
        batch_size (int, optional): _description_. Defaults to 100.
        n_parse_threads (int, optional): _description_. Defaults to 4.

    Returns:
        dataset.batch: batch dataset object 
    """
    dataset = tf.data.TextLineDataset(file).skip(1)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

def preprocess(record):
    """Preprocessing method of a dataset object, one-hot-encodes the data

    Args:
        record (_type_): _description_

    Returns:
        X (2D np.array): one-hot-encoded input sequence
        Y (1D np.array): MPRA measurements for each cell state
    """
    # Get fields from the data
    fields = tf.io.decode_csv(record, record_defaults=record_defaults)
    
    # One-hot-encode data
    chars = tf.strings.bytes_split(fields[0])
    chars_indices = table.lookup(chars)
    
    # Create a mask for out-of-vocabulary characters
    oov_mask = tf.equal(chars_indices, len(vocab))
    
    # Explicitly cast the value to int64
    chars_indices = tf.where(oov_mask, tf.constant(len(vocab), dtype=tf.int64), chars_indices)
    
    X = tf.one_hot(chars_indices, depth = len(vocab))
    #X = replace_zeros_with_random_one(X)

    # Combine y for each cell type into one vector 
    Y = tf.stack(fields[2:])
    
    # Replace missing values with -1
    Y= tf.where(tf.equal(Y,  "nan"), ["-1"], Y)
    Y= tf.where(tf.equal(Y,  "NA"), ["-1"], Y)
    Y = tf.strings.to_number(Y, tf.float32)
    return X,Y

# Get first item of the dataset to get the shape of the input data
for element in data_reader("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/randomized_motifs_no_upper_LibA_wide_pivot_all_states.csv"):
    input_shape = element[0].shape
    output_shape = element[1].shape
    
print(output_shape)
print(input_shape)

(92, 7)
(92, 262, 4)


In [43]:
# CROSS VALIDATION (10 fold)
from sklearn.model_selection import train_test_split, KFold

whole_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/no_upper_LibA_wide_pivot_all_states.csv"
out_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/"
# Split the data in two partitions
whole_data = pd.read_csv(whole_data)
k = 10
kf = KFold(n_splits = k, shuffle = True, random_state = 2008)

o=1
# For each fold we split again to get the third partition
for i in kf.split(whole_data):
    
    # Extract data from fold
    train = whole_data.iloc[i[0]]
    test =  whole_data.iloc[i[1]]
    
    # Split into train, validation and test
    valid, validation = train_test_split(train, test_size=0.111, random_state=42)
    
    # Randomize motifs
    #train = randomize_motifs_and_augment(train, num_augmentations=100)
    #train["rnd_seq"] = train['rnd_seq'].str.lower() 
    #test = randomize_motifs_and_augment(test, num_augmentations=100)
    #test["rnd_seq"] = test['rnd_seq'].str.lower() 
    #validation = randomize_motifs_and_augment(validation, num_augmentations=100)
    #validation["rnd_seq"] = validation['rnd_seq'].str.lower() 
    
    train.astype(str).to_csv(out_data+"randommotif_all_states_no_upper_CV"+str(o)+"_LibA_wide_pivot_state3_train.csv", index=False)
    test.astype(str).to_csv(out_data+"randommotif_all_states_no_upper_CV"+str(o)+"_LibA_wide_pivot_state3_test.csv", index=False)
    validation.astype(str).to_csv(out_data+"randommotif_all_states_no_upper_CV"+str(o)+"_LibA_wide_pivot_state3_validation.csv", index=False)
    o+=1
    

In [41]:
# Here we initialize the df where each fold test prediction will be appended to
# the list containing the correlations of each fold is also initialized
corr_list = []

# We define a custom normalization layer to then compile on the model
class CustomNormalization(Layer):
    """Custom normalization layer that normalizes the output of the neural network"""
    def __init__(self, **kwargs):
        super(CustomNormalization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Add trainable variables for mean and standard deviation
        self.mean = self.add_weight("mean", shape=(1,), initializer="zeros", trainable=True)
        self.stddev = self.add_weight("stddev", shape=(1,), initializer="ones", trainable=True)
        super(CustomNormalization, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        # Normalize the inputs using the learned mean and standard deviation
        return (inputs - self.mean) / (self.stddev + 1e-8)

# We define the method to compute the pearson correlation between prediction and ground truth in the multi_head case
def pearson_correlation_multi_head(predictions, ground_truth, mask_value=-1):
    """Computes Pearson Correlation between predictions and ground truth for each column
    Args:
        predictions (np.array): 2D array of prediction values (N, 7)
        ground_truth (np.array): 2D array of ground truth values (N, 7)
        mask_value (float): Value in ground truth to be ignored in correlation computation

    Returns:
        correlations (np.array): 1D array of Pearson correlations for each column
    """
    # Ensure predictions and ground_truth have the same shape
    assert predictions.shape == ground_truth.shape, "Input shapes do not match"

    n_columns = predictions.shape[1]
    correlations = np.zeros(n_columns)

    for col in range(n_columns):
        x = predictions[:, col]
        y = ground_truth[:, col]

        # Exclude values in ground truth equal to mask_value
        valid_indices = (y != mask_value)
        x = x[valid_indices]
        y = y[valid_indices]

        if len(x) == 0 or len(y) == 0:
            # If no valid values, set correlation to NaN
            correlations[col] = np.nan
        else:
            # Calculate mean of x and y
            mean_x = np.mean(x)
            mean_y = np.mean(y)

            # Calculate the numerator and denominators of the correlation coefficient
            numerator = np.sum((x - mean_x) * (y - mean_y))
            denominator_x = np.sqrt(np.sum((x - mean_x) ** 2))
            denominator_y = np.sqrt(np.sum((y - mean_y) ** 2))

            # Calculate the correlation coefficient
            correlation = numerator / (denominator_x * denominator_y)
            correlations[col] = correlation

    return correlations



import matplotlib.pyplot as plt
%matplotlib inline
# Define plotting function of loss
def create_plots(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()
    

In [46]:
from tensorflow.keras.optimizers import Adam

corr_coefficients = {}
df_test_10folds  = pd.DataFrame(columns=['State_3E',
                                         "seq",
                                         "avg_prediction_State_1M",
                                         "avg_prediction_State_2D",
                                         "avg_prediction_State_3E",
                                         "avg_prediction_State_4M",
                                         "avg_prediction_State_5M",
                                         "avg_prediction_State_6N",
                                         "avg_prediction_State_7M"])
# We iterate through each of the train folds to train, test and validate the model

for i in range(1,11):
    #Define inputs
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/randommotif_all_states_no_upper_CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/randommotif_all_states_no_upper_CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/randommotif_all_states_no_upper_CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
    
    # Read test data to then predict
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    
    predictions_sum = np.zeros((df_test.shape[0], 7))
    
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
    
    correlation_ensemble = []
    # Initialize vector containing predictions
    
    for i in range(1,11):
        
        tf.random.set_seed(i)
        
        # Define and compile model
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.5)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.5)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.5)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.5)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(7, activation='linear')(layer)
        norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

        model = Model(inputs=inputs, outputs=norm_predictions)
        model.summary()

        # compile model
        model.compile(optimizer= Adam(clipvalue=1.0),  # You can adjust the clipvalue as needed
                    loss="mse",
                    metrics=["mse"]
                    )

        # Run model
        history=model.fit(data_reader(input_path_train, batch_size=100),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=500),
                                callbacks=None,
                                verbose=1)    

        # We predict the test data
        predicted = model.predict(data_reader(input_path_test, batch_size=500))

        # Sum the predicted values (will be averaged at the end of the 10 iterations)
        predictions_sum =  np.add(predictions_sum, predicted)
        
        
        # We reed the data in the same order to compute the correlation score
        test_data = data_reader(input_path_test,batch_size=500)
        test_tensor = np.empty(shape=[0,7])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1], axis=0)
            
        # Append correlation coefficient and append to previous
        corr_coefficient = pearson_correlation_multi_head(test_tensor, predicted, mask_value=-1)
        correlation_ensemble.append(corr_coefficient)
        avg_corr_coefficient = 1
        
        
    # We fill the dataframe with predictions and fold annotation
    
    predicted = predictions_sum/10 # We divide by the number of iterations
    
    df_test["avg_prediction_State_1M"] = predicted[:,0]
    df_test["avg_prediction_State_2D"] = predicted[:,1]
    df_test["avg_prediction_State_3E"] = predicted[:,2]
    df_test["avg_prediction_State_4M"] = predicted[:,3]
    df_test["avg_prediction_State_5M"] = predicted[:,4]
    df_test["avg_prediction_State_6N"] = predicted[:,5]
    df_test["avg_prediction_State_7M"] = predicted[:,6]
    df_test["fold"] = str(i)
    df_test["partition"] = "test"
    corr_coefficients["fold"+str(i)] = avg_corr_coefficient

    
    # Append fold to previous folds
    df_test_10folds = pd.concat([df_test_10folds, df_test], ignore_index=True)    
    

# Save the results for all folds
df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/ensemble_multihead_random_motif_background_rndomdna_LibA_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)

model_path = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/background_rndomdna_test_multihead_last_cv.h5"
model.save_weights(model_path, save_format='h5') 

df_test_10folds

Model: "model_105"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 262, 4)]          0         
                                                                 
 conv1 (Conv1D)              (None, 256, 250)          7250      
                                                                 
 dropout_525 (Dropout)       (None, 256, 250)          0         
                                                                 
 batch_normalization_420 (B  (None, 256, 250)          1000      
 atchNormalization)                                              
                                                                 
 conv2 (Conv1D)              (None, 249, 250)          500250    
                                                                 
 batch_normalization_421 (B  (None, 249, 250)          1000      
 atchNormalization)                                      

2023-11-23 12:35:14.334628: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15120925269580436490


77/77 [==============================] - 2s 24ms/step - loss: 0.1787 - mse: 0.1787 - val_loss: 0.1946 - val_mse: 0.1946
Epoch 3/20
77/77 [==============================] - 2s 25ms/step - loss: 0.1728 - mse: 0.1728 - val_loss: 0.1866 - val_mse: 0.1866
Epoch 4/20
77/77 [==============================] - 2s 25ms/step - loss: 0.1702 - mse: 0.1702 - val_loss: 0.1812 - val_mse: 0.1812
Epoch 5/20
77/77 [==============================] - 2s 23ms/step - loss: 0.1663 - mse: 0.1663 - val_loss: 0.1812 - val_mse: 0.1812
Epoch 6/20
77/77 [==============================] - 2s 25ms/step - loss: 0.1652 - mse: 0.1652 - val_loss: 0.1791 - val_mse: 0.1791
Epoch 7/20
77/77 [==============================] - 2s 24ms/step - loss: 0.1632 - mse: 0.1632 - val_loss: 0.1815 - val_mse: 0.1815
Epoch 8/20
77/77 [==============================] - 2s 24ms/step - loss: 0.1597 - mse: 0.1597 - val_loss: 0.1719 - val_mse: 0.1719
Epoch 9/20
77/77 [==============================] - 2s 25ms/step - loss: 0.1585 - mse: 0.1585 

2023-11-23 12:35:50.176255: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16945764344238418800


Model: "model_106"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 262, 4)]          0         
                                                                 
 conv1 (Conv1D)              (None, 256, 250)          7250      
                                                                 
 dropout_530 (Dropout)       (None, 256, 250)          0         
                                                                 
 batch_normalization_424 (B  (None, 256, 250)          1000      
 atchNormalization)                                              
                                                                 
 conv2 (Conv1D)              (None, 249, 250)          500250    
                                                                 
 batch_normalization_425 (B  (None, 249, 250)          1000      
 atchNormalization)                                      

...

2023-11-23 13:43:58.093382: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14054784032406147676
2023-11-23 13:43:58.093480: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7982088150852318554
2023-11-23 13:43:58.093505: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8342765490410530238


77/77 [==============================] - 5s 25ms/step - loss: 0.2078 - mse: 0.2078 - val_loss: 0.1955 - val_mse: 0.1955
Epoch 2/20
 4/77 [>.............................] - ETA: 1s - loss: 0.1935 - mse: 0.1935

2023-11-23 13:43:58.455096: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14054784032406147676
2023-11-23 13:43:58.455164: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3173052644599110806
2023-11-23 13:43:58.455181: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15120925269580436490


77/77 [==============================] - 2s 22ms/step - loss: 0.1764 - mse: 0.1764 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 3/20
77/77 [==============================] - 2s 22ms/step - loss: 0.1716 - mse: 0.1716 - val_loss: 0.1918 - val_mse: 0.1918
Epoch 4/20
77/77 [==============================] - 2s 22ms/step - loss: 0.1677 - mse: 0.1677 - val_loss: 0.1893 - val_mse: 0.1893
Epoch 5/20
77/77 [==============================] - 2s 23ms/step - loss: 0.1659 - mse: 0.1659 - val_loss: 0.1873 - val_mse: 0.1873
Epoch 6/20
77/77 [==============================] - 2s 23ms/step - loss: 0.1634 - mse: 0.1634 - val_loss: 0.1852 - val_mse: 0.1852
Epoch 7/20
77/77 [==============================] - 2s 25ms/step - loss: 0.1606 - mse: 0.1606 - val_loss: 0.1791 - val_mse: 0.1791
Epoch 8/20
77/77 [==============================] - 2s 23ms/step - loss: 0.1590 - mse: 0.1590 - val_loss: 0.1661 - val_mse: 0.1661
Epoch 9/20
77/77 [==============================] - 2s 22ms/step - loss: 0.1559 - mse: 0.1559 

,State_3E,seq,avg_prediction_State_1M,avg_prediction_State_2D,avg_prediction_State_3E,avg_prediction_State_4M,avg_prediction_State_5M,avg_prediction_State_6N,avg_prediction_State_7M,CRS,State_1M,State_2D,State_4M,State_5M,State_6N,State_7M,fold,partition
0,-0.003656,aggaccggatcaactaaacaactcaaacaagggctaatataaccca...,0.013293,0.077724,0.106830,-0.045632,-0.453197,-0.051921,-0.438073,LibA.Seq7829,0.220825,0.091926,0.069263,NaN,-0.184409,NaN,10,test
1,0.163263,aggaccggatcaactaaacactagtcatacttaaaaattgcaagga...,-0.237121,-0.095708,0.071451,-0.363371,-0.796820,-0.408782,-0.768702,LibA.Seq271,0.113779,0.063990,0.075333,NaN,0.060112,NaN,10,test
2,-0.059611,aggaccggatcaactaaacaggttctgacgtatgctcctctatgga...,0.000835,0.048089,0.084202,-0.068798,-0.582745,-0.109126,-0.560780,LibA.Seq4548,-0.091269,0.008748,-0.032317,NaN,-0.123168,NaN,10,test
3,-0.060434,aggaccggatcaactaaacccgagcctgcctagccctagcttctct...,0.034859,0.069137,0.103883,-0.033280,-0.438245,-0.044886,-0.412887,LibA.Seq4582,0.011752,-0.038761,0.014523,0.051586,-0.036038,0.068024,10,test
4,0.255777,aggaccggatcaactaaacggagcagagttagtgtcaggtcaaaaa...,0.021559,0.061552,0.116517,-0.060263,-0.675636,-0.084806,-0.682032,LibA.Seq2863,0.269027,0.181167,0.099999,NaN,0.175921,NaN,10,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8487,-0.052776,aggaccggatcaacttttggtcggttgacggtcgccttgattattc...,0.027442,0.078571,0.111220,-0.044754,-0.481436,-0.052500,-0.496638,LibA.Seq4154,-0.012530,-0.020680,-0.033009,-0.013301,-0.118172,-0.027401,10,test
8488,0.254768,aggaccggatcaactttttcagtgaaagatcaccgcgggatctcac...,-0.042266,0.087214,0.138986,-0.167260,-0.532804,-0.208513,-0.531298,LibA.Seq8531,0.320753,0.342002,0.172941,0.249537,0.382781,0.211203,10,test
8489,-0.067905,aggaccggatcaacttttttagtaaaactcttaaacagtgattaca...,-0.467714,-0.224035,0.067738,-0.601953,-0.986905,-0.646018,-0.976370,LibA.Seq6744,0.073526,0.130104,0.203765,NaN,NaN,NaN,10,test
8490,0.113337,aggaccggatcaacttttttatctggttatcattctagtctagtgc...,-0.137702,-0.027394,0.077931,-0.247960,-0.694656,-0.294387,-0.663603,LibA.Seq1298,0.090932,0.186463,0.107309,NaN,0.218181,0.209876,10,test


In [55]:
print(test_tensor)
predicted

[[ 0.00353613  0.07518419 -0.01655716 ... -1.         -0.11134089
  -1.        ]
 [-1.         -1.         -0.00298327 ... -1.         -1.
  -1.        ]
 [-0.07137636 -0.06134903 -0.00907898 ...  0.11418133 -0.05733551
   0.06713761]
 ...
 [ 0.07352568  0.13010429 -0.06790534 ... -1.         -1.
  -1.        ]
 [ 0.09093156  0.18646304  0.11333688 ... -1.          0.21818118
   0.2098759 ]
 [ 0.31706724  0.26511219  0.24284895 ...  0.24410093  0.39600435
   0.05305091]]


array([[-0.02290465,  0.02176369,  0.09784307, ..., -0.50695731,
        -0.1132434 , -0.48265616],
       [-0.06179093,  0.01357291,  0.09251511, ..., -0.69948775,
        -0.22608565, -0.67693599],
       [-0.16922193, -0.01276091,  0.08360926, ..., -0.77339633,
        -0.35206431, -0.77027264],
       ...,
       [-0.46771446, -0.2240352 ,  0.06773763, ..., -0.98690457,
        -0.64601812, -0.97636985],
       [-0.13770193, -0.02739391,  0.07793133, ..., -0.69465593,
        -0.29438733, -0.66360279],
       [-0.24427834, -0.07738609,  0.07973632, ..., -0.86939697,
        -0.49781748, -0.85458084]])